# CATBOOST

In [2]:
!pip install catboost -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 7.9 MB/s eta 0:00:00


In [9]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/"
!pip install -q kaggle
!mkdir kaggle
!cp kaggle.json kaggle/
!ls kaggle
!chmod 600 kaggle/kaggle.json
!kaggle datasets list
print("Trying to download now")
!kaggle datasets download -d artem2203/chemistry-data

mkdir: cannot create directory ‘kaggle’: File exists
kaggle.json
ref                                                              title                                              size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
rahulvyasm/netflix-movies-and-tv-shows                           Netflix Movies and TV Shows                         1MB  2024-04-10 09:48:38          21054        465  1.0              
jainaru/world-happiness-report-2024-yearly-updated               World Happiness Report- 2024                       62KB  2024-05-15 19:04:34           1625         33  1.0              
sahirmaharajj/school-student-daily-attendance                    School Student Daily Attendance                     2MB  2024-04-29 19:29:56           5526        115  1.0              


In [10]:
!unzip chemistry-data.zip

Archive:  chemistry-data.zip
  inflating: 19_35000.xlsx           
  inflating: data_descriptors_transformer.csv  
  inflating: descriptor_data.csv     
  inflating: morganFP_rdkit.csv      
  inflating: rdkit_2d.csv            


In [12]:
import pandas as pd

In [13]:
df=pd.read_csv("/content/morganFP_rdkit.csv")
df

,Unnamed: 0,Title,IC50,SMILES,bit_id_0,bit_id_1,bit_id_2,bit_id_3,bit_id_4,bit_id_5,...,bit_id_2038,bit_id_2039,bit_id_2040,bit_id_2041,bit_id_2042,bit_id_2043,bit_id_2044,bit_id_2045,bit_id_2046,bit_id_2047
0,0,CHEMBL2206459,0.000015,[H]\N=C(N)\N[C@@H](C1)[C@@H](NC(=O)C)[C@@H](C=...,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,1,CHEMBL3818159,0.000016,O=C(O)C1=C[C@H](N)[C@@H](NC(=O)C)[C@@H](C1)COC...,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,CHEMBL1956716,0.000033,C=CC(=O)N[C@H]([C@H](C1)N)[C@@H](C=C1C([O-])=O...,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,CHEMBL1956715,0.000032,CC(=O)N[C@H]([C@H](C1)N)[C@@H](C=C1C([O-])=O)O...,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,CHEMBL4444029,0.000041,O=C(O)c1c(O)c(=O)cc([nH]1)-c(c2C)ccc(c2)-c3noc...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36372,36372,CHEMBL3699085,2366.790000,CC(=O)N[C@H]1CC[C@@H]([C@@H]12)[C@@H](O[Si](C)...,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
36373,36373,CHEMBL109004,2372.100000,CC(=O)Nc(cc1)c(OC(=O)C)cc1C(=O)O,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36374,36374,CHEMBL2259758,2600.970000,c1cccc(c12)cccc2CNC(=O)CCCCCCO[C@]3(C(=O)O)C[C...,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
36375,36375,CHEMBL109781,2793.000000,NCCCC(=O)Nc(c(cc1)NC(=O)C)cc1C(=O)O,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [92]:
# df=pd.read_csv("gdrive/MyDrive/db/morganFP_rdkit.csv")
# df=pd.read_csv("gdrive/MyDrive/db/descriptor_data.csv")
df=pd.read_csv("/content/rdkit_2d.csv")
df=df[300:36000]
data=df.drop(['Title', 'SMILES'], axis=1)
# cols=data.columns[:42]
# data=data[cols]
data=data.dropna()
nan_counts = data.isnull().sum()
nan_counts.sum()


0

In [93]:
def remove_correlated_columns(df, threshold=0.9):
    """
    Удаляет коррелирующие столбцы из датафрейма, оставляя только один столбец из каждой группы
    высококоррелированных столбцов.

    Параметры:
    df (pandas.DataFrame): Входной датафрейм.
    threshold (float): Порог корреляции (по умолчанию 0.9).

    Возвращает:
    pandas.DataFrame: Датафрейм с удаленными коррелирующими столбцами.
    """
    corr_matrix = df.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

    return df.drop(to_drop, axis=1)


In [94]:
# Удалим коррелирующие столбцы
df_cleaned = remove_correlated_columns(data)

In [95]:
import math

y=data['IC50']
data=df_cleaned.drop(['IC50', 'Unnamed: 0', 'NumRadicalElectrons', 'SMR_VSA8', 'SlogP_VSA9', 'fr_diazo', 'fr_isocyan', 'fr_isothiocyan', 'fr_prisulfonamd'], axis=1)
from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(
#     data.values,
#     y.values,
#     test_size=0.2,
#     shuffle=True)


X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)



In [96]:
X_train.shape

(24987, 166)

In [108]:
!pip install optuna -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 9.3 MB/s eta 0:00:00


In [109]:
import optuna
from warnings import filterwarnings
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

filterwarnings("ignore", category=DeprecationWarning)
filterwarnings("ignore", category=FutureWarning)
filterwarnings("ignore", category=UserWarning)

In [110]:
def objective(trial):
    param = {}
    param['learning_rate'] = trial.suggest_discrete_uniform("learning_rate", 0.001, 0.01, 0.004)
    param['depth'] = trial.suggest_int('depth', 5, 11)
    param['l2_leaf_reg'] = trial.suggest_discrete_uniform('l2_leaf_reg', 1.0, 5.0, 0.5)
    param['min_child_samples'] = trial.suggest_categorical('min_child_samples', [1, 4, 8, 16, 32])
    param['grow_policy'] = 'Depthwise'
    param['iterations'] = 8000
    param['use_best_model'] = True
    param['eval_metric'] = 'RMSE'
    param['od_type'] = 'iter'
    param['od_wait'] = 50
    param['random_state'] = 42
    param['logging_level'] = 'Silent'
    param['task_type'] ='GPU'

    regressor = CatBoostRegressor(**param)
    regressor.fit(X_train.copy(), y_train.copy(),
                  eval_set=[(X_val.copy(), y_val.copy())],
                  early_stopping_rounds=100)
    return check_rmse(regressor, X_val,y_val)


In [111]:
def check_rmse(model, x_val,y_val):
    pred = model.predict(x_val)
    return np.sqrt(mean_squared_error(y_val,pred))

In [ ]:
%%time
study = optuna.create_study(study_name='catboost-seed')
study.optimize(objective, n_trials=10000, timeout=24000)

[I 2024-05-20 12:14:09,018] A new study created in memory with name: catboost-seed
[I 2024-05-20 12:15:44,620] Trial 0 finished with value: 5.482941689277361 and parameters: {'learning_rate': 0.001, 'depth': 7, 'l2_leaf_reg': 2.5, 'min_child_samples': 1}. Best is trial 0 with value: 5.482941689277361.
[I 2024-05-20 12:16:43,274] Trial 1 finished with value: 5.388199647224845 and parameters: {'learning_rate': 0.005, 'depth': 9, 'l2_leaf_reg': 5.0, 'min_child_samples': 32}. Best is trial 1 with value: 5.388199647224845.
[I 2024-05-20 12:17:08,054] Trial 2 finished with value: 5.409072078277636 and parameters: {'learning_rate': 0.009, 'depth': 6, 'l2_leaf_reg': 4.5, 'min_child_samples': 1}. Best is trial 1 with value: 5.388199647224845.
[I 2024-05-20 12:18:36,794] Trial 3 finished with value: 5.525426313083197 and parameters: {'learning_rate': 0.001, 'depth': 7, 'l2_leaf_reg': 5.0, 'min_child_samples': 1}. Best is trial 1 with value: 5.388199647224845.
[I 2024-05-20 12:20:52,018] Trial 4 

In [ ]:
# {'learning_rate': 0.009, 'depth': 11, 'l2_leaf_reg': 5.0, 'min_child_samples': 1}

In [102]:
import catboost as cb
from sklearn.metrics import mean_squared_error, r2_score
model = cb.CatBoostRegressor(iterations=4000, learning_rate=0.05,depth=5,loss_function='RMSE',eval_metric='RMSE', verbose=100)
model.fit(X_train,y_train, eval_set=(X_val, y_val))


0:	learn: 7.2657955	test: 6.8196000	best: 6.8196000 (0)	total: 50.2ms	remaining: 3m 20s
100:	learn: 6.2350721	test: 6.0083447	best: 6.0083447 (100)	total: 4.35s	remaining: 2m 47s
200:	learn: 5.9291459	test: 5.8328700	best: 5.8328700 (200)	total: 6.29s	remaining: 1m 58s
300:	learn: 5.6659535	test: 5.7147721	best: 5.7147721 (300)	total: 7.95s	remaining: 1m 37s
400:	learn: 5.4567845	test: 5.6419968	best: 5.6419968 (400)	total: 9.63s	remaining: 1m 26s
500:	learn: 5.2710310	test: 5.5877812	best: 5.5876232 (499)	total: 11.3s	remaining: 1m 19s
600:	learn: 5.1269250	test: 5.5495932	best: 5.5495932 (600)	total: 13s	remaining: 1m 13s
700:	learn: 4.9870316	test: 5.5267047	best: 5.5255549 (698)	total: 14.7s	remaining: 1m 9s
800:	learn: 4.8624937	test: 5.5035576	best: 5.5033619 (799)	total: 18s	remaining: 1m 11s
900:	learn: 4.7568127	test: 5.4866814	best: 5.4857897 (893)	total: 20.5s	remaining: 1m 10s
1000:	learn: 4.6632464	test: 5.4764329	best: 5.4762142 (993)	total: 22.2s	remaining: 1m 6s
1100:	l

In [105]:
y_pred = model.predict(X_test)

# Оценка MSE и R-квадрат
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [106]:
r2

0.3736626680668873

In [44]:
import xgboost as xgb

# Создание объекта матрицы данных DMatrix
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Задание параметров
params = {
    'max_depth': 3,  # максимальная глубина дерева
    'eta': 0.1,  # скорость обучения
    'objective': 'reg:squarederror',  # функция потерь для задачи регрессии
    'eval_metric': 'rmse',  # метрика для оценки качества
    'subsample': 0.8,  # доля обучающих примеров для построения дерева
    'colsample_bytree': 0.7,  # доля признаков для построения дерева
}

# Обучение модели
num_boost_round = 4000  # количество итераций бустинга
model = xgb.train(params, dtrain, num_boost_round, evals=[(dtest, 'Test')], early_stopping_rounds=10, verbose_eval=100)

# Оценка модели на тестовых данных
y_pred = model.predict(dtest)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'RMSE на тестовых данных: {rmse:.4f}')

[0]	Test-rmse:1.14285
[67]	Test-rmse:1.13516
RMSE на тестовых данных: 1.1352


# NetBiTE

In [20]:
!pip install rdkit -q

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score


# from google.colab import drive
# drive.mount('/content/gdrive')
import pandas as pd


df=pd.read_csv("/content/rdkit_2d.csv")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class MoleculeDataset(Dataset):
    def __init__(self, smiles, ic50):
        self.smiles = smiles
        self.ic50 = ic50

    def __len__(self):
        return len(self.smiles)

    def __getitem__(self, idx):
        mol = Chem.MolFromSmiles(self.smiles[idx])
        features = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)
        features = np.array(features, dtype=np.float32)

        return {
            'features': torch.tensor(features),
            'ic50': torch.tensor(self.ic50[idx])
        }

class NetBiTE(nn.Module):
    def __init__(self, num_features, hidden_size=128):
        super(NetBiTE, self).__init__()
        self.fc1 = nn.Linear(num_features, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, 1)
        self.relu = nn.ReLU()

    def forward(self, features):
        x = self.relu(self.fc1(features))
        x = self.relu(self.fc2(x))
        output = self.fc3(x)
        return output

def train_model(model, dataloader, optimizer, criterion, epochs=10):
    for epoch in range(epochs):
        model.train()
        for batch in dataloader:
            features = batch['features'].to(device)
            ic50 = batch['ic50'].to(device)

            optimizer.zero_grad()
            output = model(features)
            loss = criterion(output.squeeze(), ic50)
            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}')

def test_model(model, dataloader, criterion):
    model.eval()
    predictions = []
    true_values = []
    with torch.no_grad():
        for batch in dataloader:
            features = batch['features'].to(device)
            ic50 = batch['ic50'].to(device)

            output = model(features)
            predictions.extend(output.squeeze().cpu().tolist())
            true_values.extend(ic50.cpu().tolist())

    mse = mean_squared_error(true_values, predictions)
    r2 = r2_score(true_values, predictions)
    print(f'MSE: {mse:.4f}, R2: {r2:.4f}')

# Загрузка данных
df=df[300:36000]
smiles = list(df['SMILES'])
ic50 = list(df['IC50'])

smiles_train, smiles_test, ic50_train, ic50_test = train_test_split(
    smiles, ic50, test_size=0.2, random_state=42
)

train_dataset = MoleculeDataset(smiles_train, ic50_train)
test_dataset = MoleculeDataset(smiles_test, ic50_test)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

model = NetBiTE(num_features=2048).to(device)

optimizer = optim.Adam(model.parameters())
criterion = nn.MSELoss()

train_model(model, train_dataloader, optimizer, criterion, epochs=10)

test_model(model, test_dataloader, criterion)

Epoch 1/10, Loss: 40.6285
Epoch 2/10, Loss: 7.7912
Epoch 3/10, Loss: 40.0827
Epoch 4/10, Loss: 14.9247


KeyboardInterrupt: 